In [6]:
## Import necessary modules for this project
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint
import string

## Opening and saving OSM XML file for auditing and cleaning
osm_file = open(r"C:\Users\jonat\Desktop\Open Street Maps project\map.osm", encoding="utf8")

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
street_types = defaultdict(set)

expected = ["Street","Avenue","Boulevard","Drive","Court","Acres","Alley",
            "Place","Way","Circle","Square","Lane","Road","Trail","Parkway","Crescent",
            "Farmdale","Forest","Heights","Ludlow","Mound","Paix","Park",
            "Pike","Riverview","Run","Terrace","Warner"]

mapping = {"St": "Street","Str": "Street","St.": "Street",
           "Ave": "Avenue",
           "Rd.": "Road",
           "Cir": "Circle"}


def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print("%s: %d") % (k, v)
        

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
            

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def update_street_name(name, mapping):
    
    name = string.capwords(name)
    m = street_type_re.search(name)
    
    if m:
        street_type = m.group()
        if street_type not in expected and street_type in mapping:
            name = re.sub(street_type_re, mapping[street_type], name)
    else:
        print("Odd Street Name: " % name)
        

# Custom class for auditing data

street_types = defaultdict(set)

def audit(osm_file):
    
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])    
    pprint.pprint(dict(street_types))

        
        
# create a dictionary of tags with tag name as key and quantity of each tags as value
        
def count_tags(osm_file):
    tags = {}
        
    for event, elem in  ET.iterparse(osm_file): 
        if elem.tag in tags:
            tags[elem.tag] += 1
        else:
            tags[elem.tag] = 1
            
    return tags

        
def iter_parse():

    tags = count_tags(osm_file)
    pprint.pprint(tags)
        

def improve_street_name(): #FUNCTION NEEDS FIXING, ERRORS WHEN RUN
    st_types = audit(osm_file)
    
    
    for st_type, ways in st_types.items():
        for name in ways:
            new_street_name = update_street_name(name, mapping)
            print(name, "changed to be: ", new_street_name)
    pprint.pprint(dict(st_types))
        
if __name__ == "__main__":
    #iter_parse()
    #osm_file.seek(0) #takes you back to begining of the dataset
    #audit(osm_file)
    improve_street_name() #FUNCTION NEEDS FIXING, ERRORS WHEN RUN

{'26': {'Jefferson Ave, #26'},
 'A': {'7703 Montgomery Road, Suite A'},
 'Ave': {'4106 Hamilton Ave',
         'Hamilton Ave',
         'Kenard Ave',
         'Paxton Ave',
         'W Clifton Ave',
         'Whetsel Ave'},
 'Cir': {'Beech View Cir'},
 'East': {'Martin Luther King Drive East'},
 'Pavillion': {'University Pavillion'},
 'Rd.': {'Montgomery Rd.'},
 'St': {'Chickasaw St', 'Vine St'},
 'West': {'West Martin Luther King Drive West'},
 'avenue': {'Hamilton avenue'}}


AttributeError: 'NoneType' object has no attribute 'items'

### Problems with the data

- Inconsistently used street names
    - I.E. 'St.', 'St', and 'Str' all used for 'Street'
    - Some streets have cardinal directions as their ending (which is fine)
- Some addresses ended with a unit number
    - I.E. '#26', 'A'
- 

In [ ]:
Notes Section (lost bits of code)

"""

"""

